# 使用 DSPy 开发分类功能案例

## 一 为 DSPy 指定模型

In [2]:
import dspy

lm = dspy.LM('ollama_chat/qwen2.5:0.5b', api_base='http://localhost:11434', api_key='', cache=False)
dspy.configure(lm=lm)

## 二 分类测试

In [3]:
from typing import Literal

class Category(dspy.Signature):
    """Classify food"""

    food: str = dspy.InputField()
    category: Literal["未知","蔬菜","水果","肉类"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Category)

result = classify(food="猪肉")
result

Prediction(
    category='肉类',
    confidence=0.85
)

In [4]:
# 打印对话记录
dspy.inspect_history()





[2025-02-05T16:18:59.474342]

System message:

Your input fields are:
1. `food` (str)

Your output fields are:
1. `category` (Literal[未知, 蔬菜, 水果, 肉类])
2. `confidence` (float)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## food ## ]]
{food}

[[ ## category ## ]]
{category}        # note: the value you produce must be one of: 未知; 蔬菜; 水果; 肉类

[[ ## confidence ## ]]
{confidence}        # note: the value you produce must be a single float value

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Classify food


User message:

[[ ## food ## ]]
猪肉

Respond with the corresponding output fields, starting with the field `[[ ## category ## ]]` (must be formatted as a valid Python Literal[未知, 蔬菜, 水果, 肉类]), then `[[ ## confidence ## ]]` (must be formatted as a valid Python float), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## category ## ]]
肉类

[[ ## confidence ## ]]
0.85



## 三 优化 Prompts

In [5]:
import csv
from dspy.evaluate import Evaluate
from dspy.teleprompt import *

In [6]:
# 加载训练集
trainset = []
with open('food_category.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        example = dspy.Example(food=row['food'], category=row['category']).with_inputs("food")
        trainset.append(example)

In [7]:
# 定义评估指标
def validate_category(example, prediction, trace=None):
    return prediction.category == example.category

In [8]:
# Evaluate our existing function
evaluator = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)
evaluator(classify, metric=validate_category)

Average Metric: 31.00 / 87 (35.6%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 87/87 [04:39<00:00,  3.22s/it]

2025/02/05 16:25:41 INFO dspy.evaluate.evaluate: Average Metric: 31 / 87 (35.6%)


,food,example_category,pred_category,confidence,validate_category
0,苹果,水果,蔬菜,0.85,
1,香蕉,水果,蔬菜,0.85,
2,橙子,水果,蔬菜,0.85,
3,草莓,水果,蔬菜,0.85,
4,葡萄,水果,蔬菜,0.85,


35.63

In [59]:
# Optimize
tp = dspy.MIPROv2(metric=validate_category, auto="light")
optimized_classify = tp.compile(classify, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)
optimized_classify.save("optimized_event_classifier.json")
dspy.inspect_history()

2025/01/23 17:54:15 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 7
valset size: 69



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program-aware proposer = 19 prompt model calls
- Program Evaluation: 25 examples in minibatch * 7 batches + 69 examples in val set * 1 full evals = 244 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectatio

2025/01/23 17:54:18 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/23 17:54:18 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/01/23 17:54:18 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Bootstrapping set 1/7
Bootstrapping set 2/7


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 346.34it/s]


...

Average Metric: 9.00 / 25 (36.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 367.86it/s]

2025/01/23 17:54:19 INFO dspy.evaluate.evaluate: Average Metric: 9 / 25 (36.0%)
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 36.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 6'].
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [52.0, 0.0, 60.0, 8.0, 40.0, 60.0, 36.0]
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [44.93]
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 44.93
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: ===========================


2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 1 =====
2025/01/23 17:54:19 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 60.0) from minibatch trials...



Average Metric: 31.00 / 69 (44.9%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 385.70it/s]

2025/01/23 17:54:20 INFO dspy.evaluate.evaluate: Average Metric: 31 / 69 (44.9%)
2025/01/23 17:54:20 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [44.93, 44.93]
2025/01/23 17:54:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 44.93
2025/01/23 17:54:20 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/23 17:54:20 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/23 17:54:20 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 44.93!







[2025-01-23T17:54:20.150874]

System message:

Your input fields are:
1. `food` (str)

Your output fields are:
1. `category` (Literal[未知, 蔬菜, 水果, 肉类])
2. `confidence` (float)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## food ## ]]
{food}

[[ ## category ## ]]
{category}        # note: the value you produce must be one of: 未知; 蔬菜; 水果; 肉类

[[ ## confidence ## ]]
{confidence}        # note: the value you produce must be a single float value

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        The pipeline is designed to classify food based on its category and confidence.


User message:

[[ ## food ## ]]
竹笋

Respond with the corresponding output fields, starting with the field `[[ ## category ## ]]` (must be formatted as a valid Python Literal[未知, 蔬菜, 水果, 肉类]), then `[[ ## confidence ## ]]` (must be formatted as a valid Python float), and then ending with the marker for `[[ ## completed ## ]]`.
